In [ ]:
from ekonlpy.sentiment import MPCK

ModuleNotFoundError: ignored

In [ ]:
import pandas as pd

df_news = pd.read_csv(r'뉴스기사.csv',encoding='utf-8-sig')
df_bondreport = pd.read_csv(r'채권분석리포트.csv',encoding='utf-8-sig')
df_minutes = pd.read_csv(r'의사록.csv',encoding='utf-8-sig')

df_list = [df_news, df_bondreport, df_minutes]
concat_df = pd.concat(df_list, ignore_index=True)
concat_df.sort_values(by='date', inplace=True)
concat_df.reset_index(drop=True, inplace=True)
concat_df.to_csv(r'n-gram_자료_concat.csv', encoding='utf-8-sig', index=False)

In [ ]:
# news 기사 n-gram화

mpck = MPCK()

def make_ngram(df):
    try:
        # content(내용) 컬럼의 text N-gram화
        tokens=mpck.tokenize(df['content'])
        ngrams=mpck.ngramize(tokens)
        tokens_text=",".join(tokens)
        ngrams_text=",".join(ngrams)
        total_text=tokens_text+","+ngrams_text
    except Exception as e:
        print(e)
        total_text=""
    return total_text

df=pd.read_csv(r"뉴스기사content.csv")
mpck = MPCK()
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True,drop=True)
df['ngrams']=df.apply(make_ngram,axis=1)
df.to_csv(r"뉴스기사n-gram.csv", encoding='utf-8-sig', index=False)

긍정사전 한글자 처리

In [ ]:
# 긍정,부정사전 한글자짜리제거
df_score = pd.read_csv('ngram_score.csv')

index_list = []
for i in range(len(df_score)):
    s = df_score['ngram'][i]
    if len(s.split('/')[0]) == 1:
        index_list.append(i)
index_list

df_score.drop(index_list,inplace=True)
df_score.to_csv('ngram_score_except1.csv', encoding='utf-8-sig', index=False)

In [ ]:
df_score = pd.read_csv('ngram_score.csv')
df_score.to_csv('ngram_score_except1.csv', encoding='utf-8-sig', index=False)

의사록 한글자 처리

In [ ]:
import pandas as pd
minutes = pd.read_csv('/content/drive/MyDrive/workspace/bok/minutes_data_save(08_21).csv',encoding='utf-8-sig')
minutes.sort_values(by='date',ascending=True,inplace=True)

import numpy as np

def remove_oneletter(text):
    ngram_list = []
    for j in range(len(text.split(','))):
        ngram = text.split(',')[j]
        ngram2 = ngram.split(';')[0]
        ngram_list.append(ngram)
        letter = ngram.split('/')[0]
        letter2 = letter.split(';')[0]
        if len(letter) == 1 or len(letter2) == 1:
            # print(ngram)
            for i,v in enumerate(ngram_list) :
                if v == ngram:
                    ngram_list[i] = np.nan
    # print(ngram_list)
    removed = ','.join(ng for ng in ngram_list if ng is not np.nan)
        
    return removed

In [ ]:
from tqdm import tqdm

removed_text = []
for i in tqdm(range(len(minutes))):
    before_text = minutes['ngrams'][i]
    print(remove_oneletter(before_text))
    removed_text.append(remove_oneletter(before_text))

minutes['ngrams'] = removed_text
minutes.to_csv('ngrams_final_except1.csv',encoding='utf-8-sig',index=False)